In [1]:
import wave
from scipy.signal import spectrogram
import soundfile as sf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
import scipy

In [2]:
data, samplerate = sf.read(
    '/Users/GodSpeed/Documents/Courses/Machine Learning/Project/LibriSpeech/dev-clean/84/121123/84-121123-0004.flac')

In [11]:
print len(data)
print samplerate
#taking 20ms samples
fft_length = 0.001 * 20 * samplerate
overlap_length = 0.001 * 10 * samplerate
#print window_size
f, t, Sxx = spectrogram(data, fs = samplerate, nperseg = fft_length, 
                        noverlap = overlap_length)
print "No. of sample frequencies: ", len(f) #len(signal.spectrogram(data)[0])
print "No. of time segmnets:", len(t)
print "Spectrogram length: ", len(Sxx)
print Sxx.shape
#data is in the form [frequencies[time]]
#print type(Sxx)
#for i in range(len(t) -  1):
#    print t[i+1] - t[i]

70400
16000
No. of sample frequencies:  161
No. of time segmnets: 439
Spectrogram length:  161
(161, 439)


In [ ]:
plt.pcolormesh(t,f,Sxx)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.show()
#print Sxx[:,313]

## Output labels

In [ ]:
output_labels = {}
for i in range(26):
    output_labels[i] = chr(i+97)
output_labels[i+1] = ' '
output_labels

### Initialize weights
Following conventions in SPEECH RECOGNITION WITH DEEP RECURRENT NEURAL NETWORKS

In [13]:
#set layer sizes
layers_sizes = [len(Sxx), 200, len(output_labels)]
input_size = len(Sxx)

"""this function is to initialize wi, 
weights from input layer to first hidden layer,
in the form [input layer[hidden layer]]"""
def initialize_wx(layers_sizes):
    input_size = layers_sizes[0]
    first_hidden_layer_size = layers_sizes[1]
    e = np.sqrt(6)/np.sqrt(input_size + first_hidden_layer_size)
    wx = np.dot(np.random.rand(input_size, 
                               first_hidden_layer_size), 2*e) - e
    return wx

"""this function is to initialize wh,
weights from each hidden layer to next hidden layer
creating wh array as [layer number[input layer[output layer]]]"""
def initialize_W_hh_next(layers_sizes):
    hidden_layers_sizes = layers_sizes[1:-1]
    num_layers = len(hidden_layers_sizes)
    #return empty if only one hidden layer
    if num_layers == 1:
        return
    wh = np.zeros((num_layers - 1, max(hidden_layers_sizes),
                   max(hidden_layers_sizes[1:len(hidden_layers_sizes)])))
    for i in range(num_layers - 1):
        e = np.sqrt(6)/np.sqrt(hidden_layers_sizes[i]+hidden_layers_sizes[i+1])
        wh[i, 0:hidden_layers_sizes[i], 0:hidden_layers_sizes[i+1]] = np.dot(
            np.random.rand((hidden_layers_sizes[i]), hidden_layers_sizes[i+1]),
            2*e)-e
    return wh

"""this function is to initialize wi, 
weights from last hidden layer to output layer,
in the form [hidden layer[output layer]]"""
def initialize_wo(layers_sizes):
    last_hidden_layer_size = layers_sizes[-2]
    output_size = layers_sizes[-1]
    wo = initialize_wx([layers_sizes[-2],layers_sizes[-1]])
    return wo

"""initialize recurrent weights for hidden layer on itself"""
def initialize_wh(layers_sizes):
    hidden_layers_sizes = layers_sizes[1:-1]
    num_hidden_layers = len(hidden_layers_sizes)
    wh = np.zeros((max(hidden_layers_sizes), num_hidden_layers))
    #print w_hh_curr.shape
    #return
    for i in range(num_hidden_layers):
        e = np.sqrt(6)/np.sqrt(2*hidden_layers_sizes[0])
        wh[0:hidden_layers_sizes[i], i] = np.dot(
            np.random.rand(hidden_layers_sizes[i]), 2*e) - e
    return wh

def initialize_wc(layers_sizes):
    wc = initialize_wh(layers_sizes)
    return wc


#initialize weights from input to 1st hidden layer
w_xi = initialize_wx(layers_sizes)

#initialize w_hi
w_hi = initialize_wh(layers_sizes)

#initialize w_ci
w_ci = initialize_wc(layers_sizes)

#initialize w_xf
w_xf = initialize_wx(layers_sizes)

#initialize w_hf
w_hf = initialize_wh(layers_sizes)

#initialize w_cf
w_cf = initialize_wc(layers_sizes)

#initialize w_xc
w_xc = initialize_wx(layers_sizes)

#initialize w_hc
w_hc = initialize_wh(layers_sizes)

#initialize w_xo
w_xo = initialize_wx(layers_sizes)

#initialize w_ho
w_ho = initialize_wh(layers_sizes)

#initialize w_co
w_co = initialize_wc(layers_sizes)


#initialize weights from each hidden layer to next hidden layer
wh_next = initialize_W_hh_next(layers_sizes)

#initialize weights from last hidden layer to output layer
w_hy = initialize_wo(layers_sizes)

#initialize recurrent weights to 0
#w_hh_curr = initialize_w_hh_curr(layers_sizes)

#print w_hi.shape
#print w_xi.shape
#print wh_next.shape
#print w_hy.shape
#print w_hh_curr.shape

Initialize h(t-1) and c(t-1) to 0

In [14]:
def initialize_empty_state(layers_sizes):
    hidden_layers_sizes = layers_sizes[1:-1]
    num_hidden_layers = len(hidden_layers_sizes)
    empty_state = np.zeros((max(hidden_layers_sizes), num_hidden_layers))
    return empty_state

hidden_state_prev = initialize_empty_state(layers_sizes)
cell_state_prev = initialize_empty_state(layers_sizes)
print hidden_state_prev.shape
print cell_state_prev.shape

(200, 1)
(200, 1)


In [15]:
print Sxx.shape
print len(t)
print len(f)

(161, 439)
439
161


### initialize bias

In [23]:
e = np.sqrt(6)/np.sqrt(2*layers_sizes[1])
print np.dot(np.random.rand(5), 2*e) - e
#bias_i, bias_o, bias_f, bias_c, bias_y = 

[-0.08098789  0.08417584  0.03637687 -0.05497636 -0.06230177]


## RNN architecture

In [49]:
class Model(object):
    def sigmoid(z):
        """z: net input from previous layer
        a: sigmoid activation of net input"""
        a = 1.0/(1.0+np.exp(-z))
        return a
    
    def tanh(z):
        """z: net input from previous layer
        a: tanh activation of input"""
        a = (np.exp(-z) - np.exp(-z))/(np.exp(-z) + np.exp(-z))
        return a
    
    def feedforward(mini_batch):
        """mini_batch: mini batch for training is to be done
        return: input_gate: input gate values for the mini batch
                forget_gate: forget gate values for the mini batch
                cell_state_gate: cell state for the mini batch
                output_gate: output gate values for the mini batch
                hidden_state: hidden state values for the mini batch
                y: output"""
        input_gate = sigmoid(np.dot(np.transpose(w_xi), xi)
                             + (w_hi * hidden_state_prev) 
                             + (w_ci * cell_state_prev) + bias_i)
        forget_gate = sigmoid(np.dot(np.transpose(w_xf), xi) + 
                              (w_hf * hidden_state_prev) + 
                              (w_cf * cell_state_prev) + bias_f)
        cell_state_gate = (forget_gate * cell_state_prev) + (input_gate * tanh(np.dot(
                    np.transpose(w_xc), xi) + (w_hc * hidden_state_prev) + bias_c))
        output_gate = sigmoid(np.dot(np.transpose(w_xo), xi) 
                              + (w_ho * hidden_state_prev) 
                              + (w_co * cell_state_gate) + bias_o)
        hidden_state = output_gate * tanh(cell_state_gate)
        y = np.dot(np.transpose(w_hy), hidden_state) + bias_y
        return input_gate, forget_gate, cell_state_gate, output_gate, hidden_state, y
    
    #def CTC(y):
        
    
    def rnn_lstm_model_train(input_data, output_data, epochs = 10):
        """input_data: sorted input data, by length, in the form of spectrograms
        output_data: output transcripts for the audio files
        mini_batch_size: size of the mini batch
        epochs: no. of iterations on the dataset"""
        for epoch in epochs:
            for iteration in number_iterations:
                x, y, text, input_lengths, label_lengths = datagen.iterate_dev(
                    50, sortBy_duration = True)
                input_gate, forget_gate, cell_state_gate, output_gate, 
                    hidden_state, y = feedforward(x)
                

#return {
#            'x': x,  # features(padded with zero) shape(minibatch-size, time-steps, feature-dimensions)
#            'y': y,  # Flattened output labels transformed as integers
#            'texts': texts,  # original list of texts
#            'input-lengths': input_lengths,  # length of each input
#            'label-lengths': label_lengths  # length of each label
#        }
#for i, batch in enumerate(datagen.iterate_dev(50, sortBy_duration=True)):
#    print i
#    print batch

(161, 10)
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 